In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 14.1 MB/s 
     |████████████████████████████████| 596 kB 76.4 MB/s 
     |████████████████████████████████| 6.6 MB 78.2 MB/s 
     |████████████████████████████████| 77 kB 7.8 MB/s 
     |████████████████████████████████| 880 kB 90.2 MB/s 
     |████████████████████████████████| 96 kB 6.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.50-py3-none-any.whl size=895166 sha256=e5b2220352709ad17fceb18d9db024df43ad11d598cca1d3d8733f1e7b95d226
  Stored in directory: /root/.cache/pip/wheels/d9/72/54/519f0d5143cc6c73fa3297509123c86fc8586a7fdea8d25311
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into 

In [ ]:
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer
from typing import Dict, List
import json

from transformers import (
    EncoderDecoderModel,
    GPT2Tokenizer as BaseGPT2Tokenizer,
    PreTrainedTokenizer,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Trainer
)
from torch.utils.data import Dataset, DataLoader
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_path = '/content/drive/MyDrive/project3/'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
limit_alphabet = 4000
vocab_size = 30522

# 모델 불러오기

Bleu로 성능을 확인할 모델을 우선 불러오고 제대로 동작하는지 확인하자. 

모델 불러오기

In [ ]:
model = EncoderDecoderModel.from_pretrained('/content/drive/MyDrive/project3/dump/models/best_model(final_koje)_v2')

In [ ]:
model.to(device)
model.eval()

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [ ]:
tokenizer = BertTokenizer(drive_path + f'dump/wpm-vocab-extend-{vocab_size}.txt', do_lower_case=False)
#src_tokenizer = BertTokenizer('/content/drive/MyDrive/project3/dump/주형준_vocab/wpm-vocab-je.txt', do_lower_case=False)
#trg_tokenizer = BertTokenizer('/content/drive/MyDrive/project3/dump/주형준_vocab/wpm-vocab-ko.txt', do_lower_case=False)

### 모델 예측값 만들기

데이터셋 확인

### 정답 예측값 구하기

In [ ]:
# jit 데이터 사용 시
src_file_path = drive_path + 'jit/ko.test'
trg_file_path = drive_path + 'jit/je.test'

data = []
data_answer = []
with open(src_file_path, 'r') as fd:
    for line in fd.readlines():
        data.append(line[:-1])
with open(trg_file_path, 'r') as fd:
    for i, line in enumerate(fd.readlines()):
        data_answer.append(line[:-1])

len(data), len(data_answer)

(5000, 5000)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class DatasetRetriever(Dataset):
    def __init__(self, features):
        super(DatasetRetriever, self).__init__()
        self.features = features

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):   
        feature = self.features[index]
        embedding = tokenizer(feature, truncation=True, max_length=512, return_attention_mask=False, return_token_type_ids=False)
        return {'input_ids' : embedding['input_ids']}

In [ ]:
test_dataset = DatasetRetriever(data)

In [ ]:
test_dataset[0]

{'input_ids': [2,
  5903,
  5774,
  13310,
  14507,
  6783,
  7,
  5774,
  1919,
  17161,
  5979,
  7,
  5843,
  5828,
  11815,
  3150,
  11815,
  1990,
  3149,
  3]}

In [ ]:
collator = DataCollatorForSeq2Seq(tokenizer, model)

test_loader = DataLoader(
    test_dataset, 
    batch_size=128, 
    shuffle=False,
    collate_fn=collator,
    num_workers=1)

In [ ]:
from tqdm import tqdm

def get_prediction(text):
    embeddings = tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
    embeddings.to(device)
    output = model.generate(**embeddings, max_length=256, bos_token_id=tokenizer.cls_token_id, eos_token_id=tokenizer.sep_token_id)[0, 1:-1].cpu()
    return tokenizer.decode(output[1:])

def get_predictions(data):
    result = []
    for batch in tqdm(data):
        batch = {key: value.to(device) for key, value in batch.items()}
        output = model.generate(batch['input_ids'], max_length=256).cpu()
        try:
            outputs = [tokenizer.decode(item[2:item.tolist().index(tokenizer.sep_token_id)]) for item in output]
        except ValueError:
            outputs = [tokenizer.decode(item[2:]) for item in output]
        result.extend(outputs)
    return result

In [ ]:
get_prediction('어서 오세요')

'ᄒᆞᆫ저 옵서'

In [ ]:
test_pred = get_predictions(test_loader)

100%|██████████| 40/40 [01:32<00:00,  2.32s/it]


In [ ]:
len(test_pred)

5000

In [ ]:
import pickle

with open('best_model_je_test_pred.pkl', 'wb') as f:
    pickle.dump(test_pred, f)

In [ ]:
test_pred2 = test_pred

In [ ]:
data[:10]

['그래도 그거 살살 떠서 먹지 . 그거 안 먹진 않아 . 어떻게 옛날 삶이야 삶이 었어',
 '마찬가지 그렇게 쌀 넣어서 물 넣고 해서 삶아가면 익어서 죽 되는 거 .',
 '그것보고 저 거시기 , 부엌 .',
 '예 , 동김치 그거 .',
 '그렇게 해버리니까 많이 김매야겠군 요',
 '그때도 가면 그땐 그렇게해서 놀았어 . 나 그거 잊어버리지 않아 . 여기 넷 다섯 , 저쪽에 넷 다섯 서면 , 딱 손 이어서 저쪽으로 가면서 무슨 사꾸라또 만또민또 하나이세 몸메 하면서 아이 하나 가서 딱 지키면 그 아이 하나 잡아오고 또 그쪽에서 와서 그렇게 하고 . 그렇게 하면서도 놀고 . 그저 밤에 진또리 .',
 '표선리란 마을이요 관광도로의 요충지가 됩니다 , 여기가 .',
 '청묵도 메밀가루 , 그냥 묵도 메밀가루',
 '그것이 구강이야 . 거기 이렇게 놨다가 이젠 삼월 나가면 요 방만큼만 예를 들어서요 방만큼만 놓아 , 씨를 . 씨를 놓으면 그것이 커 가면 그때도 오줌도 주고 무엇 조금 줘 . 빨리 크라고 . 그러면 크면은 유월절 , 유월절 안 와도 저 땅만 마르지 않으면 줄도 길고 하면 그냥 잘라서 가서 고랑 만들어 놓았다가 심어 .',
 '병아리 깨워서 병아리 크면 .']

In [ ]:
test_pred[:10]

['경헤도 그거 ᄉᆞᆯᄉᆞᆯ 거령 먹주게. 그거 안 먹진 안허여. 어떵 엿날 삶이사 삶이엇어',
 '마찬가지 경 ᄊᆞᆯ 놩 물 놓곡 헹 ᄉᆞᆱ아가민 익엉 죽 뒈는 거.',
 '그것ᄀᆞ라 저 거시기, 정지.',
 '예, 동짐치 그거.',
 '경 해부난 많이 검질메야겟다예.',
 '그때도 가면은 그땐 경 헨 놀앗저. 나 그거 잊어불지 안허여. 이디 넷 다섯, 저쪽에 넷 다섯 사민, 딱 손 이어근에 저쪽으로 가멍 무신 사꾸라또 만또민또 ᄒᆞ나이세 몸메 허멍 아이 ᄒᆞ나 강 딱 지키민 가이 ᄒᆞ나 잡아오고 또 그쪽에서 오랑 경허곡. 경 허멍도 놀고. 그자 밤이 진또리.',
 '표선리란 부락예 관광도로의 요충지가 됩니다, 여기가.',
 '청묵도 ᄆᆞ물ᄏᆞ루, 그냥 묵도 모ᄆᆞᆯᄏᆞ루.',
 '그것이 구강이야. 그디 영 놧당 이젠 삼월 나가민 요 방만이만 예를 들엉예 방만이만 놩, 씨를. 씨를 놓민 그것이 커 가민 그때도 오줌도 주고 무시거 ᄒᆞ꼼 줘. 제기 크렌. 게민 크민 유월절, 유월절 안 오라도 저 땅만 ᄆᆞᆯ르지 안허민 줄도 질고 허민 그냥 ᄍᆞᆯ랑 강 고랑 만들어 놧당 싱거.',
 '빙에기 께왕 빙에기 크민.']

### 정답값 구하기.

In [ ]:
data_answer[:10]

['경헤도 거 ᄉᆞᆯᄉᆞᆯ 거려뒁 먹주 . 거 안 먹진 안허여 . 어떵 옛날 삶사 삶 이랏어',
 '마찬가지 겅 ᄊᆞᆯ 넣어근엥 물 넣곡 허영 ᄉᆞᆱ아가면 익어근엥 죽 뒈는 거',
 '그것ᄀᆞ란 저 거세기 , 정지 .',
 '예 , 동김치 . 그거 .',
 '겅 헤부난 많이 검질메야겠다예 .',
 '그때도 가민 그땐 경 헨 놀앗어 . 나 그거 잊어불지 안 허여 . 이디 넷 다섯 저짝에 넷 다섯 사민 , 딱 손 이어근에 저레 가멍 무신 사꾸라또 민또만또 하나이세 몸메 허멍 아이 ᄒᆞ나 강 딱 지키민 그 아이 ᄒᆞ나 심어오고 또 그쪽에서 오랑 그추룩 허곡 . 경 허멍도 놀고 . 그자 밤이 진또리 .',
 '표선리란 마을이예 , 관광도로의 요충지가 뒙니다 , 여기가 .',
 '청묵도 ᄆᆞ물ᄏᆞ루 , 그냥 묵도 ᄆᆞ물ᄏᆞ루 .',
 '그것이 구강이라이 . 그디 영 놧당 이젠 삼월 나가민 또 요 구들만이만 예를 들러서 요 구들만이만 놓아 , 씨를 . 씨를 놓면 것이 키와가민 그때도 오줌도 주곡이 무시거 ᄒᆞ꼼 주어 . 재기 크렌 . 경허민 크면은 유월절 , 유월절 아니 겪어도 저 땅만 ᄆᆞ르지 안허면은이 줄도 질곡 허면은 그냥 끈엉 강 고랑 청 놧당 심어 .',
 '빙애기 꾀왕 빙애기 크민 .']

In [ ]:
with open('best_model_ko_data_answer.pkl', 'wb') as f:
    pickle.dump(data_answer, f)

# Bleu score 구하기.

In [ ]:
import nltk

hypothesis = test_pred[0]
reference = data_answer[0]
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(BLEUscore)

0.573048798376626


In [ ]:
BLEUscore_sum = []
for i in range(len(test_pred)):
    hypothesis = test_pred[i]
    reference = data_answer[i]
    BLEU = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
    BLEUscore_sum.append(BLEU)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
avg = sum(BLEUscore_sum, 0.0) / len(BLEUscore_sum)
print(avg)

0.6112880811830279


In [ ]:
len(test_pred)

5000

#Bleu score(JIT Testset)

\\
### 제주어 to 표준어
    1. only jit : 71.9
    2. jit + ai_hub : 59.7
    3. jit + preprocessed ai_hub : 74.3
    4. jit + preprocessed ai_hub_v2 :  74.3
### 표준어 to 제주어
    1. only jit : 59.0
    2. jit + ai_hub : 41.3
    3. jit + preprocessed ai_hub : 60.7
    4. jit + preprocessed ai_hub_v2 :  61.1